In [1]:
import numpy as np
import math
import random as rand

In [2]:
rand.seed()
rand.randint(0,5)

1

In [3]:
#small_df = pd.read_csv('CS205_small_testdata__3.txt', sep="  ",header=None)
#large_df = pd.read_csv('CS205_large_testdata__36.txt', sep="  ",header=None)

small = np.loadtxt("CS205_small_testdata__3.txt")
large= np.loadtxt('CS205_large_testdata__36.txt')

In [4]:
# This is a testing stub to make sure that the search is working correctly
def acc_class(ftr):
    return(rand.randint(0,50))

In [5]:
acc_class(2)

44

In [6]:
for j in range(1,len(small[1,:])): print(j)

1
2
3
4
5
6
7
8
9
10


In [7]:
def ftr_search(mat):
    # List to keep track of all the current features
    crnt_ftr = []
    levels_best = []
    
    # Each level of the search tree
    for i in range(1,len(mat[1,:])):
        # List to keep track of the added feature
        add_ftr = -1
        # Keeps track of the best accuracy
        best_acc = -1
        print("On the", i, "th level of the search tree")
        
        # At each level, we want to see which features we can add
        for j in range(1,len(mat[1,:])):
            #print(j)
            #print(crnt_ftr)
            # At each level, I want to test the accuracy of adding a certain feature
            # but I don't want to consider a feature that's already included
            if j not in crnt_ftr:
                print("  Considering adding the ", j, "feature")
                # Calculate the accuracy for the given set of features
                acc = knn(mat, crnt_ftr, j)
                #acc = acc_class(j)
                # Checking if we should add this feature based on how it compares
                # to the best accuracy so far.
                #print(best_acc,acc)
                if acc > best_acc:
                    best_acc = acc
                    add_ftr = j # This is always updated when the accuracy is better.
                #print(add_ftr)
                
        # After we check all the features that we can add at this level
        # we want to make the current feature set is updated based on the best feature   
        crnt_ftr.append(add_ftr) 
        # The best combination of features and its accuracy at the current level
        levels_best.append((best_acc,crnt_ftr[:]))  # [:] is a copy of what is in the list now
        #print(add_ftr)
        print("*On level ", i, "added feature", add_ftr, "to current set*")
        #print(crnt_ftr)
        #print(crnt_ftr)
        #print(len(set(crnt_ftr)))
    return(levels_best)

In [8]:
# Creating a function that will perform kNN
def knn(mat, crnt_set_, new_ftr):
    '''
    '''
    # Keep track of accuracy
    acc = 0
    # The features that will be used for classification
    crnt_set = crnt_set_[:]
    crnt_set.append(new_ftr)
    #print(crnt_set)
  
    # Leave one out cross validation for each of the observations
    # with K = 1 nearest neighbors
    for i in range(0,len(mat)):
        # The observation to classify
        obs = mat[i,crnt_set]
        obs_label = mat[i,0]
        # An initial value for the distance that will be updated
        # if the distance at some iteration is smaller than the previous 
        # smallest distance.
        nn_dist = 1000000
        # Variable to keep track of the index that's closest to obs.
        nn_ind = 0
        # Now we compare each observation to the rest
        for j in range(0,len(mat)):
            # A check to make sure that the function is working properly
            # print("Ask if", i, "is nearest neighbor with", j)
            if j != i: # don't compare observation to itself
                #print(len(obs),len(mat[j,crnt_set]))
                #print(obs, mat[j,crnt_set])
                dist = math.sqrt(sum(obs-mat[j,crnt_set])**2)
                #print(dist,nn_dist)
                if dist < nn_dist: # update label if this is true
                    nn_dist = dist
                    nn_ind = j
                    label = mat[j,0] # the label of the closest observation
                    
        # After finding the best label, check whether it was classified correctly
        if label == obs_label:
            acc = acc + 1
            
    acc_rate = acc/len(mat)
    return(acc_rate)

In [9]:
# Testing the ftr_search function with the small dataset
ftr_search(small)

On the 1 th level of the search tree
  Considering adding the  1 feature
  Considering adding the  2 feature
  Considering adding the  3 feature
  Considering adding the  4 feature
  Considering adding the  5 feature
  Considering adding the  6 feature
  Considering adding the  7 feature
  Considering adding the  8 feature
  Considering adding the  9 feature
  Considering adding the  10 feature
*On level  1 added feature 4 to current set*
On the 2 th level of the search tree
  Considering adding the  1 feature
  Considering adding the  2 feature
  Considering adding the  3 feature
  Considering adding the  5 feature
  Considering adding the  6 feature
  Considering adding the  7 feature
  Considering adding the  8 feature
  Considering adding the  9 feature
  Considering adding the  10 feature
*On level  2 added feature 1 to current set*
On the 3 th level of the search tree
  Considering adding the  2 feature
  Considering adding the  3 feature
  Considering adding the  5 feature
  Con

[(0.84, [4]),
 (0.74, [4, 1]),
 (0.75, [4, 1, 5]),
 (0.76, [4, 1, 5, 9]),
 (0.7466666666666667, [4, 1, 5, 9, 3]),
 (0.7366666666666667, [4, 1, 5, 9, 3, 2]),
 (0.71, [4, 1, 5, 9, 3, 2, 6]),
 (0.7166666666666667, [4, 1, 5, 9, 3, 2, 6, 7]),
 (0.7, [4, 1, 5, 9, 3, 2, 6, 7, 8]),
 (0.7133333333333334, [4, 1, 5, 9, 3, 2, 6, 7, 8, 10])]

In [13]:
import time
start_time = time.time()
print(start_time)
# Testing the ftr_search function with the large dataset
#ftr_search(large)
#print("--- %s seconds ---" % (time.time() - start_time)/60)

1622832115.594087


In [293]:
print(combs)
combs.sort()   # Sort by the first element in the tuple first
print(combs)


[(40, [4]), (42, [4, 6]), (38, [4, 6, 3]), (50, [4, 6, 3, 5]), (36, [4, 6, 3, 5, 0]), (40, [4, 6, 3, 5, 0, 1]), (46, [4, 6, 3, 5, 0, 1, 2]), (44, [4, 6, 3, 5, 0, 1, 2, 9]), (39, [4, 6, 3, 5, 0, 1, 2, 9, 8]), (11, [4, 6, 3, 5, 0, 1, 2, 9, 8, 10]), (28, [4, 6, 3, 5, 0, 1, 2, 9, 8, 10, 7])]
[(11, [4, 6, 3, 5, 0, 1, 2, 9, 8, 10]), (28, [4, 6, 3, 5, 0, 1, 2, 9, 8, 10, 7]), (36, [4, 6, 3, 5, 0]), (38, [4, 6, 3]), (39, [4, 6, 3, 5, 0, 1, 2, 9, 8]), (40, [4]), (40, [4, 6, 3, 5, 0, 1]), (42, [4, 6]), (44, [4, 6, 3, 5, 0, 1, 2, 9]), (46, [4, 6, 3, 5, 0, 1, 2]), (50, [4, 6, 3, 5])]


In [104]:
# testing the kNN \function on the small dataset with all featuers
knn(mat = small, crnt_set, new_ftr)

0.7133333333333334

In [74]:
int(sum(small[1,1:]-small[2,1:]))**2

25

In [262]:
sum(small[1,[1,2]]-small[2,[1,2]])**2

4.456618812189467

In [105]:
small[1,[1,2,3]]

array([ 0.02937927,  0.75751174, -0.57226014])

In [110]:
len(small[1,-0])

TypeError: object of type 'numpy.float64' has no len()

In [108]:
for i in range(0,10): print(i)

0
1
2
3
4
5
6
7
8
9


In [111]:
small[1,-0]

2.0

In [134]:
l.append(1)

AttributeError: 'int' object has no attribute 'append'

In [316]:
for j in range(,5): print(j)

0
1
2
3
4
